In [36]:
#%pip install lightgbm
#%pip install catboost

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns #Visualização
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool


In [38]:
df_train = pd.read_csv('train.csv').drop(columns='index')
df_test = pd.read_csv('test.csv').drop(columns='index')

In [39]:
df_train.head()

,CS_SEXO,NU_IDADE_N,CS_GESTANT,CS_RACA,CS_ESCOL_N,SG_UF,CO_RG_RESI,FEBRE,TOSSE,GARGANTA,...,SIND_DOWN,HEPATICA,DIABETES,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,FADIGA,VACINA_COV,EVOLUCAO
0,M,71,6,1,3.0,SP,1331.0,2.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,0.0
1,M,53,6,4,3.0,SP,1335.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0
2,M,70,6,1,1.0,SC,1546.0,1.0,2.0,2.0,...,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,9.0,1.0
3,M,60,6,9,1.0,MG,1452.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0
4,M,61,6,4,1.0,SP,1342.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,0.0


In [40]:
df_test.head()

,CS_SEXO,NU_IDADE_N,CS_GESTANT,CS_RACA,CS_ESCOL_N,SG_UF,CO_RG_RESI,FEBRE,TOSSE,GARGANTA,...,CARDIOPATI,SIND_DOWN,HEPATICA,DIABETES,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,FADIGA,VACINA_COV
0,M,63,6,1,NaN,SP,1348.0,1.0,1.0,2.0,...,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,NaN
1,F,52,5,4,9.0,MG,1448.0,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
2,F,45,5,1,NaN,SP,1351.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F,46,5,4,4.0,SP,1346.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
4,F,27,5,4,3.0,GO,1779.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0


In [41]:
df_test.columns

Index(['CS_SEXO', 'NU_IDADE_N', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF',
       'CO_RG_RESI', 'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP',
       'SATURACAO', 'DIARREIA', 'VOMITO', 'FATOR_RISC', 'CARDIOPATI',
       'SIND_DOWN', 'HEPATICA', 'DIABETES', 'PNEUMOPATI', 'IMUNODEPRE',
       'RENAL', 'OBESIDADE', 'FADIGA', 'VACINA_COV'],
      dtype='object')

In [42]:
# sns.pairplot(df_test[['NU_IDADE_N', 'CS_GESTANT', 'CS_ESCOL_N', 'SG_UF',
#        'FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP',
#        'SATURACAO', 'DIARREIA', 'VOMITO', 'FATOR_RISC', 'CARDIOPATI',
#        'SIND_DOWN', 'HEPATICA', 'DIABETES', 'PNEUMOPATI', 'IMUNODEPRE',
#        'RENAL', 'OBESIDADE', 'FADIGA', 'VACINA_COV']].sample(100), hue='VACINA_COV')

In [43]:
df_train.nunique()

CS_SEXO         3
NU_IDADE_N    120
CS_GESTANT      8
CS_RACA         6
CS_ESCOL_N      7
SG_UF          27
CO_RG_RESI    319
FEBRE           3
TOSSE           3
GARGANTA        3
DISPNEIA        3
DESC_RESP       3
SATURACAO       3
DIARREIA        3
VOMITO          3
FATOR_RISC      2
CARDIOPATI      3
SIND_DOWN       3
HEPATICA        3
DIABETES        3
PNEUMOPATI      3
IMUNODEPRE      3
RENAL           3
OBESIDADE       3
FADIGA          3
VACINA_COV      3
EVOLUCAO        2
dtype: int64

In [44]:
# X = df_train.drop("EVOLUCAO", axis=1)
# y = df_train["EVOLUCAO"]
X = df_train.select_dtypes(['number']).fillna(0).drop(columns='EVOLUCAO')
y = df_train['EVOLUCAO']

X_test = df_test.select_dtypes(['number']).fillna(0)

In [45]:
cat_feature = str(list(range(0, X.shape[1])))

print(cat_feature)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [46]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=12)

In [47]:
trein_data = Pool(
    data= X_train,
    label=y_train
)

model = CatBoostClassifier(iterations=100)
model.fit(trein_data)
preds_class = model.predict(trein_data)

Learning rate set to 0.5
0:	learn: 0.6028686	total: 760ms	remaining: 1m 15s
1:	learn: 0.5816637	total: 1.18s	remaining: 57.9s
2:	learn: 0.5733810	total: 1.65s	remaining: 53.4s
3:	learn: 0.5684138	total: 1.98s	remaining: 47.4s
4:	learn: 0.5666163	total: 2.22s	remaining: 42.2s
5:	learn: 0.5643274	total: 2.56s	remaining: 40.1s
6:	learn: 0.5626629	total: 2.79s	remaining: 37s
7:	learn: 0.5600328	total: 3.09s	remaining: 35.5s
8:	learn: 0.5589130	total: 3.38s	remaining: 34.2s
9:	learn: 0.5571661	total: 3.84s	remaining: 34.6s
10:	learn: 0.5560789	total: 4.08s	remaining: 33s
11:	learn: 0.5543118	total: 4.38s	remaining: 32.1s
12:	learn: 0.5537924	total: 4.64s	remaining: 31.1s
13:	learn: 0.5533851	total: 4.96s	remaining: 30.4s
14:	learn: 0.5526503	total: 5.59s	remaining: 31.7s
15:	learn: 0.5522507	total: 5.84s	remaining: 30.6s
16:	learn: 0.5516081	total: 6.38s	remaining: 31.1s
17:	learn: 0.5511776	total: 6.82s	remaining: 31.1s
18:	learn: 0.5505246	total: 7.04s	remaining: 30s
19:	learn: 0.5500984	

In [48]:
preds_class

array([1., 1., 1., ..., 0., 0., 0.])

In [49]:
model.score(X_val,y_val)

0.7279932396370753

In [50]:
pd.DataFrame(preds_class).reset_index().to_csv('out_lr.csv', header=['Id', 'EVOLUCAO'], index=False)